In [1]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import matplotlib.pyplot as plt
import math as mt
import tensorflow as tf
import tabula as tb
from tabula import read_pdf
fp_padrao = 0.92
fp_padrao_Sup = 2-fp_padrao

In [2]:
def normaliza(s):
    if s >= 0:
       return s
    elif s < 0 :
       return s + 2

universal01_dataset = pd.read_excel('universal.xlsx', 'Universal_01')
universal02_dataset = pd.read_excel('universal.xlsx', 'Universal_02')
univ01 = universal01_dataset
univ02 = universal02_dataset

Universal = pd.concat([univ02,univ01])

Universal.drop(columns=['Freqüência', 'Ocorrência'],inplace=True)

Universal.rename(columns ={'Fat,Pot T': 'FP T', 
                            'Fat,Pot, R': 'FP R',
                            'Fat,Pot, S': 'FP S'}, inplace=True)
Universal_p = pd.pivot_table(Universal, index=['Data'])

Universal_p['Pot R KVA'] = (Universal_p['Corrente R'] * Universal_p['Tensão R'] * (mt.sqrt(3)))/1000
Universal_p['Pot S KVA'] = (Universal_p['Corrente S'] * Universal_p['Tensão S'] * (mt.sqrt(3)))/1000
Universal_p['Pot T KVA'] = (Universal_p['Corrente T'] * Universal_p['Tensão T'] * (mt.sqrt(3)))/1000

#Calculo KVAr usando numpy
Universal_p['Pot R KW'] = (Universal_p['Pot R KVA'] * (Universal_p['FP R']))
Universal_p['Pot S KW'] = (Universal_p['Pot S KVA'] * (Universal_p['FP S']))
Universal_p['Pot T KW'] = (Universal_p['Pot T KVA'] * (Universal_p['FP T']))

#Calculo KVAr usando numpy
Universal_p['Pot R KVAr'] = np.sqrt(np.power(Universal_p['Pot R KVA'],2) - np.power(Universal_p['Pot R KW'],2))
Universal_p['Pot S KVAr'] = np.sqrt(np.power(Universal_p['Pot S KVA'],2) - np.power(Universal_p['Pot S KW'],2))
Universal_p['Pot T KVAr'] = np.sqrt(np.power(Universal_p['Pot T KVA'],2) - np.power(Universal_p['Pot T KW'],2))

#Calculo Faltante Kvar
Universal_p['Pot R KVAr faltante'] = (Universal_p['Pot R KVAr']) - (np.sqrt(np.power(Universal_p['Pot R KVA'],2)-
                                                                      (np.power((Universal_p['Pot R KVA'] * (fp_padrao)),2))))
Universal_p['Pot S KVAr faltante'] = (Universal_p['Pot S KVAr']) - (np.sqrt(np.power(Universal_p['Pot S KVA'],2)-
                                                                      (np.power((Universal_p['Pot S KVA'] * (fp_padrao)),2))))
Universal_p['Pot T KVAr faltante'] = (Universal_p['Pot T KVAr']) - (np.sqrt(np.power(Universal_p['Pot T KVA'],2)-
                                                                      (np.power((Universal_p['Pot T KVA'] * (fp_padrao)),2))))

Universal["FP R"] = Universal["FP R"].apply(normaliza)
Universal["FP S"] = Universal["FP S"].apply(normaliza)
Universal["FP T"] = Universal["FP T"].apply(normaliza)

Universal_p["FP R"] = Universal_p["FP R"].apply(normaliza)
Universal_p["FP S"] = Universal_p["FP S"].apply(normaliza)
Universal_p["FP T"] = Universal_p["FP T"].apply(normaliza)

fpfases_abaixo = Universal.loc[((Universal["FP R"]< fp_padrao) | (Universal["FP R"]> fp_padrao_Sup))
                         | ((Universal["FP S"]< fp_padrao) | (Universal["FP S"]> fp_padrao_Sup))
                         | ((Universal["FP T"]< fp_padrao) | (Universal["FP T"]> fp_padrao_Sup))]

fpfases = Universal_p.loc[((Universal_p["FP R"]< fp_padrao) | (Universal_p["FP R"]> fp_padrao_Sup))
                         | ((Universal_p["FP S"]< fp_padrao) | (Universal_p["FP S"]> fp_padrao_Sup))
                         | ((Universal_p["FP T"]< fp_padrao) | (Universal_p["FP T"]> fp_padrao_Sup))]

maximoR = fpfases['Pot R KVAr faltante'].max()
maximoS = fpfases['Pot S KVAr faltante'].max()
maximoT = fpfases['Pot T KVAr faltante'].max()

In [3]:
def fases():
    exceldados = pd.ExcelWriter('FP Fases.xlsx',
                                engine='xlsxwriter',
                                datetime_format='mmm-d-yyyy hh:mm:ss',
                                date_format='mmmm-dd-yyyy',)
    Universal_p.round(5).to_excel(exceldados, sheet_name='Universal') #1
    fpfases.round(5).to_excel(exceldados, sheet_name='FP Fora do padrão') #2
    exceldados.save()

    from win32com.client import Dispatch
    appexcel = Dispatch('Excel.Application')
    wb = appexcel.Workbooks.Open('J:\\LEO\\Universal Plasticos\\FP Fases.xlsx')
    for i in range (1, 3, 1):
        appexcel.Worksheets(i).Activate()
        appexcel.ActiveSheet.Columns.AutoFit()
    appexcel.Worksheets(1).Activate()
    wb.Save()
    wb.Close()


def graficos():
    exceldados = pd.ExcelWriter('FP Fases gráficos.xlsx',
                                engine='xlsxwriter',
                                datetime_format='mmm-d-yyyy hh:mm:ss',
                                date_format='mmmm-dd-yyyy',)
    Universal_p.round(5).to_excel(exceldados, sheet_name='Universal') #1
    fpfases.round(5).to_excel(exceldados, sheet_name='FP Fora do padrão') #2
    exceldados.save()

    from win32com.client import Dispatch
    appexcel = Dispatch('Excel.Application')
    wb = appexcel.Workbooks.Open('J:\\LEO\\Universal Plasticos\\FP Fases gráficos.xlsx')
    for i in range (1, 3, 1):
        appexcel.Worksheets(i).Activate()
        appexcel.ActiveSheet.Columns.AutoFit()
    appexcel.Worksheets(1).Activate()
    wb.Save()
    wb.Close()

In [4]:
fases()

In [5]:
Valores_ComparacaoR = fpfases[(fpfases['FP R'] > fpfases['FP T'] ) & (fpfases['FP R'] > fpfases['FP T'])]
Valores_ComparacaoS = fpfases[(fpfases['FP S'] > fpfases['FP T'] ) & (fpfases['FP S'] > fpfases['FP T'])]
Valores_ComparacaoT = fpfases[(fpfases['FP T'] > fpfases['FP R'] ) & (fpfases['FP T'] > fpfases['FP S'])]

In [6]:
Universal_p.round(5).to_csv('Universal Leituras.csv')
fpfases.round(5).to_csv('Universal FP FORAS.csv')